In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
def clip(img):
    return img.clip(roi)

In [ ]:
roi = Map.draw_last_feature.geometry()

In [ ]:
images = ee.ImageCollection("NASA/GIMMS/3GV0").select('ndvi')\
        .filterDate('2012', '2013').map(clip)

In [ ]:
visParam = {
 'min': -1,
 'max': 1,
 'palette': ['000000', 'f5f5f5', '119701']
}
Map.addLayer(images, visParam, 'NDVI')

In [ ]:
img = images.toBands()
ps = img.sample(region = roi, geometries = True)

In [ ]:
ps.first().getInfo()

In [ ]:
Map.addLayer(ps, {}, 'ps')

In [ ]:
order = 2
window_size = 11
def savitzky_golay(y):
    half_window = (window_size - 1) / 2
    deriv = 0
    order_range = ee.List.sequence(0, order)
    k_range = ee.List.sequence(-half_window, half_window)
    def fun1(k):
        return order_range.map(lambda o: ee.Number(k).pow(o))
    b = ee.Array(k_range.map(fun1))
    mPI = ee.Array(b.matrixPseudoInverse())
    impulse_response = (mPI.slice(**{ 'axis': 0, 'start': deriv, 'end': deriv + 1 })).project([1])
    y0 = y.get(0)
    firstvals = y.slice(1, half_window + 1).reverse().map(
        lambda e: ee.Number(e).subtract(y0).abs().multiply(-1).add(y0))
    yend = y.get(-1)
    lastvals = y.slice(-half_window - 1, -1).reverse().map(
        lambda e: ee.Number(e).subtract(yend).abs().add(yend))
    y_ext = firstvals.cat(y).cat(lastvals)
    runLength = ee.List.sequence(0, y_ext.length().subtract(window_size))
    smooth = runLength.map(
        lambda i:ee.Array(y_ext.slice(ee.Number(i), ee.Number(i).add(window_size))).multiply(impulse_response).reduce("sum", [0]).get([0])
    )
    return smooth

In [ ]:
def fun(feature):
    List = bands.map(lambda i: feature.get(i))
    sg_ = savitzky_golay(List)
    return feature.set(ee.Dictionary.fromLists(bands_sg, sg_))

In [ ]:
bands = img.bandNames()
List = bands.map(lambda i: ps.first().get(i))

In [ ]:
bands_sg = bands.map(lambda i: ee.String(i).cat('_sg'))
bands_sg.getInfo()

In [ ]:
sg_ps = ps.map(fun)

In [ ]:
img_sg = sg_ps.reduceToImage(properties=[bands_sg.get(-1)], 
                             reducer=ee.Reducer.mean())\
        .reproject(images.first().projection().crs(),
                  images.first().projection().getInfo()['transform'])
Map.addLayer(img_sg, visParam, 'sg')

In [ ]:
sg_ps.first().getInfo()

In [ ]:
bands.get(0).cat('_sg').getInfo()

In [ ]:
a = ee.Array(ps.first().get('array')).toList()

In [ ]:
sg_ps.first().getInfo()

In [ ]:
from scipy.signal import savgol_filter